In [1]:
import cv2 as cv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math
import dlib

In [2]:
detector = dlib.get_frontal_face_detector() #Face detector
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

In [3]:
data = pd.read_csv('data/fer2013.csv')

In [4]:
N = 1000
D = data
data.groupby('Usage').count()

,emotion,pixels
Usage,,
PrivateTest,3589,3589
PublicTest,3589,3589
Training,28709,28709


In [5]:
training_image = []
training_label_full = []
test_image = []
test_label_full = []


for index, row in D.iterrows():
    if row['Usage'] == 'Training':
        training_image.append(np.reshape(np.array(row['pixels'].split(), dtype=np.uint8), (48, 48)))
        training_label_full.append(row['emotion'])
    else:
        test_image.append(np.reshape(np.array(row['pixels'].split(), dtype=np.uint8), (48, 48)))
        test_label_full.append(row['emotion'])

In [6]:
def get_landmarks(_image):
    image = _image.copy()
    detections = detector(image, 1)
    for k,d in enumerate(detections): #For all detected face instances individually
        shape = predictor(image, d) #Draw Facial Landmarks with the predictor class
        xlist = []
        ylist = []
        for i in range(1,68): #Store X and Y coordinates in two lists
            xlist.append(float(shape.part(i).x))
            ylist.append(float(shape.part(i).y))
                
#         xmean = np.mean(xlist) #Get the mean of both axes to determine centre of gravity
#         ymean = np.mean(ylist)
        xmean = xlist[29]
        ymean = ylist[29]
        xcentral = [(x-xmean) for x in xlist] #get distance between each point and the central point in both axes
        ycentral = [(y-ymean) for y in ylist]
        
        # point 29 - nose tip
        # point 26 - middle point b/w two eyes
        cv.circle(image, (int(xlist[29]), int(ylist[29])), 1, (0,0,255))
        
        angle_nose = np.arctan2((ylist[26]-ymean), (xlist[26]-xmean)) * 180 / np.pi
        if angle_nose < 0:
            angle_nose += 90
        else:
            angle_nose -= 90
            
#         print(angle_nose)

        landmarks_vectorised = []
        for x, y, w, z in zip(xcentral, ycentral, xlist, ylist):
            landmarks_vectorised.append(x) #Add the coordinates relative to the centre of gravity
            landmarks_vectorised.append(y)

            #Get the euclidean distance between each point and the centre point (the vector length)
            meannp = np.asarray((ymean,xmean))
            coornp = np.asarray((z,w))
            dist = np.linalg.norm(coornp-meannp)
            landmarks_vectorised.append(dist)

            #Get the angle the vector describes relative to the image, corrected for the offset that the nosebrigde has when the face is not perfectly horizontal
            anglerelative = (np.arctan2((z - ymean), (w - xmean)) * 180 / np.pi) - angle_nose
            landmarks_vectorised.append(anglerelative)
        
    if len(detections) < 1: 
        landmarks_vectorised = "error"     
    
    return landmarks_vectorised

In [7]:
import time

def make_sets():
    training_data = []
    test_data = []
    training_label = []
    test_label = []
    
    #Append data to training and prediction list, and generate labels 0-7
    index = 0
    
    start_time = time.time()
    
    for item in training_image:
        # image = cv.imread(item) #open image
        # gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY) #convert to grayscale
        # clahe_image = clahe.apply(gray)
#         print(index)
        landmarks_vectorised = get_landmarks(item)
        if landmarks_vectorised == "error":
            pass
        else:
            training_data.append(landmarks_vectorised) #append image array to training data list
            training_label.append(training_label_full[index])
            
        index += 1
        
    print('train - ', time.time() - start_time)
    
    index = 0
    for item in test_image:
        # image = cv.imread(item)
        # gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
        # clahe_image = clahe.apply(gray)
#         print(index)
        landmarks_vectorised = get_landmarks(item)
        if landmarks_vectorised == "error":
            pass
        else:
            test_data.append(landmarks_vectorised)
            test_label.append(test_label_full[index])
            
        index += 1
        
    print('test - ', time.time() - start_time)

    return training_data, test_data, training_label, test_label

# --------------------------------------------------------

train_data, test_data, training_label, test_label = make_sets()

print(len(train_data), len(training_label))

# pd.DataFrame(train_data)
# pd.DataFrame(test_data)

('train - ', 105.0227210521698)
('test - ', 131.75428295135498)
(19863, 19863)


In [8]:
from sklearn import svm

start_time = time.time()

clf = svm.SVC(kernel='rbf', gamma=2)
clf.fit(train_data, training_label)

print('fit time - ', time.time() - start_time, ' sec')

('fit - ', 396.141273021698, ' sec')


In [10]:
start_time = time.time()
predicted_labels = clf.predict(test_data)
print('predict time - ', time.time() - start_time, ' sec')

('predict time - ', 28.47614312171936, ' sec')


In [12]:
print('Training accuracy (using kernel SVC) - ', np.mean(predicted_labels == test_label) * 100)

('Training accuracy (using kernel SVC) - ', 35.860531116967366)


In [22]:
from sklearn.neural_network import MLPClassifier

nn_clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(50, 10), random_state=1, max_iter=500)
nn_clf.fit(train_data, training_label)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(50, 10), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [23]:
start_time = time.time()
nn_predicted_labels = nn_clf.predict(test_data)
print('predict time - ', time.time() - start_time, ' sec')

('predict time - ', 0.12700581550598145, ' sec')


In [24]:
print('Training accuracy (using MLP) - ', np.mean(nn_predicted_labels == test_label) * 100)

('Training accuracy (using MLP) - ', 27.85323332657612)
